In [1]:
%pip install gradio==3.44.0 torch dice

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Genom10\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from src.dnd.units import Unit, GenericSoldier
from src.dnd.game_board import DnDBoard, MovementError
from src.gui.gridDrawer import generate_grid, draw_field
from src.gui.adapters import RenderUnit
from src.dnd.load_unit import load_unit, load_renderUnit, getTokenImagePath, getTokenName

from typing import List
import gradio as gr
import itertools
import glob
import re
import numpy as np

C:\Users\Genom10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
board_size = None
board = None
render_units: List[RenderUnit] = []
selectedToken = None
selectedCell = None


def generate_board(gridScale, gridSize):
  global board_size
  global board
  board_size = gridSize
  board = DnDBoard((board_size, board_size))
  field = generate_grid(board_size, gridScale)
  return field

def add_token(image, x:int, y:int, gridScale, jsonDescriptor = None):
  global board_size
  global selectedToken
  # print(jsonDescriptor.name)
  if jsonDescriptor is None:
    board.place_unit(GenericSoldier(), (x,y), 0)
    render_units.append(RenderUnit(board.board[(x,y)].get_UID(), (x,y), image))
  else:
    board.place_unit(load_unit(jsonDescriptor.name), (x,y), 0)
    r_unit = load_renderUnit(jsonDescriptor.name, (x,y))
    r_unit.unitUID = board.board[(x,y)].get_UID()
    render_units.append(r_unit)
  if x < board_size:
    x += 1
  else:
    x = 0
    y += 1
  # print(type(x))
  return (draw_field(render_units, selectedToken, gridScale, board_size), 
          x, 
          y, 
          update_UID_list())

def move_token(index, x:int, y:int, gridScale):
  global board_size
  global selectedToken
  unit:Unit = board.get_unit_by_UID(render_units[index].getUID())
  try:
    board._set_unit_position(unit=unit, new_position=(y,x))
    render_units[index].pos = np.array([y, x], dtype = int)
  except MovementError as e:
    print("Error moving into chosen cell:", e)
  return draw_field(render_units, selectedToken, gridScale, board_size)

def set_selected_token(tokenID, gridScale):
  global board_size
  global selectedToken
  # print('set selectedToken to:', selectedToken)
  if type(tokenID) == int:
    selectedToken = tokenID
  else:
    selectedToken = None
  return draw_field(render_units, selectedToken, gridScale, board_size)

def on_grid_scale_change(gridScale):
  global board_size
  global selectedToken
  return draw_field(render_units, selectedToken, gridScale, board_size)

def update_UID_list():
  new_choices = [x.getUID() for x in render_units]
  new_choices = list(zip(new_choices, itertools.count()))
  return gr.Dropdown.update(choices=new_choices, interactive=True, label='tokenID')

def on_board_click(boardImg, gridScale, evt: gr.SelectData):
  global board
  # print(f'{evt.index=}')
  clickedCell = (evt.index[1]//(gridScale+1), evt.index[0]//(gridScale+1))
  try:
    index = [tuple(x.getPos()) for x in render_units].index(clickedCell)
  except ValueError:
    index = None
  return index

Interface

In [4]:
with gr.Blocks() as demo:
  #Grid
  with gr.Row():
    gridSize = gr.Slider(label="gridSize", value=10, minimum=2, maximum=100, step=1)
    makeBoard = gr.Button(value="Make Board")
  gridScale = gr.Slider(label="gridScale", value=64, minimum=16, maximum=128, step=1)
  im_canvas = gr.Image(sources=[], interactive=False)
  makeBoard.click(generate_board, inputs = [gridScale, gridSize], outputs=[im_canvas])
  gridScale.change(on_grid_scale_change, inputs = [gridScale], outputs=[im_canvas])

  with gr.Tabs(visible=True) as tabs:
    #Add Token
    with gr.TabItem("Add Token") as tab1:
      with gr.Row():
        with gr.Column(min_width=10):
          x = gr.Number(label="x pos", precision=0)
          y = gr.Number(label="y pos", precision=0)
          label = gr.Text()
          jsonDescriptor = gr.File()
        im_in = gr.Image(image_mode='RGBA', type='pil')
        examples_paths = glob.glob('./Tokens/*.json')
        token_data = [[getTokenImagePath(path), getTokenName(path), path] for path in examples_paths]
        # print(token_data)
        examples = gr.Examples(examples=token_data, inputs=[im_in, label, jsonDescriptor], examples_per_page=4)
      btn_add = gr.Button(value="Add token")

    #Move Token
    with gr.TabItem("Move Token") as tab2:
      with gr.Row():
          tokenID = gr.Dropdown(['t1', 't2'], label='tokenID', allow_custom_value=True)
          tab2.select(update_UID_list, outputs=tokenID)
          x_move = gr.Number(label="x pos", precision=0)
          y_move = gr.Number(label="y pos", precision=0)
      btn_move = gr.Button(value="Move token")
    btn_add.click(add_token, inputs=[im_in, y, x, gridScale, jsonDescriptor], outputs=[im_canvas, y, x, tokenID])
    tokenID.change(set_selected_token, inputs=[tokenID, gridScale], outputs=[im_canvas])
    btn_move.click(move_token, inputs=[tokenID, x_move, y_move, gridScale], outputs=[im_canvas])
    im_canvas.select(on_board_click, inputs=[im_canvas, gridScale], outputs=[tokenID])

if __name__ == "__main__":
    demo.launch()

C:\Users\Genom10\AppData\Local\Temp\ipykernel_6920\3857505417.py:7: GradioUnusedKwargWarning: You have unused kwarg parameters in Image, please remove them: {'sources': []}
  im_canvas = gr.Image(sources=[], interactive=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
